In [1]:
%load_ext tensorboard

import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
class LeNet(Sequential):
    def __init__(self, input_shape, nb_classes):
        super().__init__()

        self.add(Conv2D(6, kernel_size=(5,5), strides= (1,1), activation = 'relu', input_shape=input_shape, padding='same')) #피처맵 개수 = 6, #padding(이미지를 0으로 둘러쌈)
        self.add(AveragePooling2D(pool_size=(2,2), strides= (2,2), padding='valid'))
        self.add(Conv2D(6, kernel_size=(5,5), strides = (1,1), activation = 'relu', input_shape=input_shape, padding='same'))
        self.add(AveragePooling2D(pool_size=(2,2), strides=(2,2),padding='valid'))
        self.add(Flatten())
        self.add(Dense(120, activation='relu'))
        self.add(Dense(84, activation='relu'))
        self.add(Dense(nb_classes, activation='softmax'))
        self.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [3]:
model = LeNet((100, 100, 3), 2)
model.summary()

Model: "le_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 6)       456       
                                                                 
 average_pooling2d (AverageP  (None, 50, 50, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 6)         906       
                                                                 
 average_pooling2d_1 (Averag  (None, 25, 25, 6)        0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 3750)              0         
                                                                 
 dense (Dense)               (None, 120)               45012

In [4]:
EPOCH = 100
BATCH_SIZE = 32
Image_height = 100
Image_width = 100
train_dir = 'data/catanddog/train'
valid_dir = 'data/catanddog/validation'

In [5]:
train = ImageDataGenerator(rescale=1./255, rotation_range = 10, width_shift_range = 0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1) #min/max scalling, 그레이 척도의 차이를 스케이링

train_generator = train.flow_from_directory(train_dir, target_size=(Image_height, Image_width), color_mode='rgb',batch_size=BATCH_SIZE, seed=1, shuffle=True, class_mode='categorical')

Found 385 images belonging to 2 classes.


In [6]:
valid = ImageDataGenerator(rescale=1.0/255.0)
valid_generator = valid.flow_from_directory(valid_dir, target_size=(Image_height, Image_width), color_mode='rgb', batch_size=BATCH_SIZE, seed=7, shuffle=True, class_mode='categorical')

train_num = train_generator.samples
valid_num = valid_generator.samples

Found 98 images belonging to 2 classes.


In [11]:
log_dir='Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [12]:
model.fit(train_generator,
    epochs=EPOCH,
    steps_per_epoch=train_num// BATCH_SIZE,
    validation_data= valid_generator,
    validation_steps= valid_num // BATCH_SIZE,
    callbacks = [tensorboard_callback],
    verbose=1)

Epoch 1/100
12/12 [==============================] - 6s 379ms/step - loss: 0.7141 - accuracy: 0.5014 - val_loss: 0.6921 - val_accuracy: 0.5000
Epoch 2/100
12/12 [==============================] - 2s 171ms/step - loss: 0.6908 - accuracy: 0.5184 - val_loss: 0.6888 - val_accuracy: 0.5521
Epoch 3/100
12/12 [==============================] - 2s 167ms/step - loss: 0.6922 - accuracy: 0.5382 - val_loss: 0.6904 - val_accuracy: 0.6146
Epoch 4/100
12/12 [==============================] - 2s 151ms/step - loss: 0.6841 - accuracy: 0.5807 - val_loss: 0.6822 - val_accuracy: 0.5729
Epoch 5/100
12/12 [==============================] - 2s 177ms/step - loss: 0.6808 - accuracy: 0.5781 - val_loss: 0.6861 - val_accuracy: 0.5521
Epoch 6/100
12/12 [==============================] - 2s 153ms/step - loss: 0.6599 - accuracy: 0.5977 - val_loss: 0.6899 - val_accuracy: 0.5625
Epoch 7/100
12/12 [==============================] - 2s 165ms/step - loss: 0.6791 - accuracy: 0.5609 - val_loss: 0.6807 - val_accuracy: 0.5938